In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
plt.rcParams.update({'font.size':16})

## Diffraction Grating Spacing Calculation ##

In [2]:
#offset calculation 
theta = (20.894444 + 21.166667)/2
offset = 21.166667 - theta
offset

0.13611150000000194

In [3]:
#errors in line measurements
e_n1r = 0.5/60
e_n1l = 1/60

e_n2r = 0.5/60
e_n2l = 1/60

In [4]:
#calculating distance between lines with accepted wavelength values and measured angles
N1r = 5889.95e-10/(np.sin(np.radians(20.894444))+ np.sin(np.radians(offset)))
N2r = 5895.924e-10/(np.sin(np.radians(20.916667))+ np.sin(np.radians(offset)))

e_N1r = 1/(np.sin(np.radians(20.894444+e_n1r))-np.sin(np.radians(20.894444)))
e_N2r = 1/(np.sin(np.radians(20.916667+e_n2r))-np.sin(np.radians(20.916667)))

In [5]:
##calculating lines per mm
d = ((N1r + N2r)/2)**-1/1000
e_d = ((e_N1r + e_N2r)/2)**-1/1000

In [6]:
d

609.5503228635721

In [7]:
e_d

1.358657235353544e-07

## Rydberg Constant For Hydrogen ##

In [8]:
H_df = pd.DataFrame({'n': [3, 4, 5], 'H_line': ['H_a', 'H_b', 'H_c'], 'primaryr': [23+29/90, 17+23/180, 15.5+1/60], 'primaryl': [360-(336+43/180), 360-(342+2/3), 360-(344+53/90)], 'secondaryr': [51+71/180, 35+43/60, 31+31/90], 'secondaryl': [360-(305+1/6), 360-(323+1/60), 360-(327+28/45)]})

In [9]:
#errors in values
e_hcpr = 2/60
e_hcpl = 1/60
e_hbpr = 2/60
e_hbpl = 0.5/60
e_hapr = 2/60
e_hapl = 1/60

e_hcsr = 1/60
e_hcsl = 2/60
e_hbsr = 0.5/60
e_hbsl = 2/60
e_hasr = 4/60
e_hasl = 0.5/60

In [10]:
#finding average angles with offset
def avg(x, y):
    return (x + y)/2

def offset(x, y):
    a = (x - y)/2
    offset = y - a
    return (x + offset)/2

H_df['prim_a'] = offset(H_df['primaryr'], H_df['primaryl'])
H_df['sec_a'] = offset(H_df['secondaryr'], H_df['secondaryl'])

e_hcp = avg(e_hcpr, e_hcpl)
e_hbp = avg(e_hbpr, e_hbpl)
e_hap = avg(e_hapr, e_hapl)

e_hcs = avg(e_hcsr, e_hcsl)
e_hbs = avg(e_hbsr, e_hbsl)
e_has = avg(e_hasr, e_hasl)

In [11]:
#converting angles to wavelengths
def wavelength(d, t, n):
    return d*np.sin(np.radians(t))/n

H_df['lambdap'] = wavelength(1/(d*1000), H_df['prim_a'], 1)
H_df['lambdas'] = wavelength(1/(d*1000), H_df['sec_a'], 2)

#errors done by taking largest value andtaking value with no error away
e_lhap = wavelength(1/((d-e_d)*1000), 23.541667+e_hap, 1)-wavelength(1/(d*1000), 23.541667, 1)
e_lhbp = wavelength(1/((d-e_d)*1000), 17.230556+e_hbp, 1)-wavelength(1/(d*1000), 17.230556, 1)
e_lhcp = wavelength(1/((d-e_d)*1000), 15.463889+e_hcp, 1)-wavelength(1/(d*1000), 15.463889, 1)

e_lhas = wavelength(1/((d-e_d)*1000), 53.113889+e_has, 1)-wavelength(1/(d*1000), 53.113889, 1)
e_lhbs = wavelength(1/((d-e_d)*1000), 36.350000+e_hbs, 1)-wavelength(1/(d*1000), 36.350000, 1)
e_lhcs = wavelength(1/((d-e_d)*1000), 31.861111+e_hcs, 1)-wavelength(1/(d*1000), 31.861111, 1)

In [12]:
#finding average wavelengths
H_df['lambda'] = (H_df['lambdap'] + H_df['lambdas'])/2

e_lha = (e_lhap + e_lhas)/2
e_lhb = (e_lhbp + e_lhbs)/2
e_lhc = (e_lhcp + e_lhcs)/2

In [13]:
#converting wavelength to rydberg constant
def ryd(n_i, n_f, l):
    a = 1/(n_f**2) - 1/(n_i**2)
    return 1/(a*l)

H_df['R_value'] = ryd(H_df['n'], 2, H_df['lambda'])

e_hra = ryd(3, 2, 6.556727e-07-e_lha)-ryd(3, 2, 6.556727e-07)
e_hrb = ryd(4, 2, 4.860760e-07-e_lhb)-ryd(4, 2, 4.860760e-07)
e_hrc = ryd(5, 2, 4.352077e-07-e_lhc)-ryd(5, 2, 4.352077e-07)

In [14]:
#find average rydberg
R_H = (1.098109e+07 + 1.097222e+07 + 1.094168e+07)/3

e_hr = (e_hra + e_hrb + e_hrc)/3

In [15]:
R_H

10964996.666666666

In [16]:
e_hr

13034.347244870538

In [17]:
H_df

,n,H_line,primaryr,primaryl,secondaryr,secondaryl,prim_a,sec_a,lambdap,lambdas,lambda,R_value
0,3,H_a,23.322222,23.761111,51.394444,54.833333,23.651389,53.973611,6.581422e-07,6.633958e-07,6.607690e-07,1.089640e+07
1,4,H_b,17.127778,17.333333,35.716667,36.983333,17.281944,36.666667,4.873658e-07,4.898354e-07,4.886006e-07,1.091553e+07
2,5,H_c,15.516667,15.411111,31.344444,32.377778,15.437500,32.119444,4.366941e-07,4.361297e-07,4.364119e-07,1.091149e+07


## Rydberg Constant for Deuterium ##

All done using the same methofds as for hydrogen

In [18]:
D_df = pd.DataFrame({'n': [3, 4, 5], 'H_line': ['H_a', 'H_b', 'H_c'], 'primaryr': [23+(19+2/3)/60, 17+(7+2/3)/60, 15+19/60], 'primaryl': [360-(336+13/60), 360-(342.5+11/60), 360-(344.5+10/60)], 'secondaryr': [51.5+(3+1/3)/60, 35.5+(12+2/3)/60, 31.5+4/60], 'secondaryl': [360-(305+10/60), 360-(323+4/60), 360-(327.5+9/60)]})

In [19]:
e_dcpr = 1/60
e_dcpl = 0.5/60
e_dbpr = 0.5/60
e_dbpl = 0.5/60
e_dapr = 0.5/60
e_dapl = 0.5/60

e_dcsr = 1/60
e_dcsl = 3/60
e_dbsr = 1/60
e_dbsl = 0.5/60
e_dasr = 0.5/60
e_dasl = 0.5/60

In [20]:
def avg(x, y):
    return (x + y)/2

def offset(x, y):
    a = (x - y)/2
    offset = y - a
    return (x + offset)/2

D_df['prim_a'] = offset(D_df['primaryr'], D_df['primaryl'])
D_df['sec_a'] = offset(D_df['secondaryr'], D_df['secondaryl'])

e_dcp = avg(e_dcpr, e_dcpl)
e_dbp = avg(e_dbpr, e_dbpl)
e_dap = avg(e_dapr, e_dapl)

e_dcs = avg(e_dcsr, e_dcsl)
e_dbs = avg(e_dbsr, e_dbsl)
e_das = avg(e_dasr, e_dasl)

In [21]:
def wavelength(d, t, n):
    return d*np.sin(np.radians(t))/n

D_df['lambdap'] = wavelength(1/(d*1000), D_df['prim_a'], 1)
D_df['lambdas'] = wavelength(1/(d*1000), D_df['sec_a'], 2)

e_ldap = wavelength(1/((d-e_d)*1000), 23.555556+e_dap, 1)-wavelength(1/(d*1000), 23.555556, 1)
e_ldbp = wavelength(1/((d-e_d)*1000), 17.222222+e_dbp, 1)-wavelength(1/(d*1000), 17.222222, 1)
e_ldcp = wavelength(1/((d-e_d)*1000), 15.32500+e_dcp, 1)-wavelength(1/(d*1000), 15.32500, 1)

e_ldas = wavelength(1/((d-e_d)*1000), 53.194444+e_das, 1)-wavelength(1/(d*1000), 53.194444, 1)
e_ldbs = wavelength(1/((d-e_d)*1000), 36.322222+e_dbs, 1)-wavelength(1/(d*1000), 36.322222, 1)
e_ldcs = wavelength(1/((d-e_d)*1000), 31.958333+e_dcs, 1)-wavelength(1/(d*1000), 31.958333, 1)

In [22]:
D_df['lambda'] = (D_df['lambdap'] + D_df['lambdas'])/2

e_lda = (e_ldap + e_ldas)/2
e_ldb = (e_ldbp + e_ldbs)/2
e_ldc = (e_ldcp + e_ldcs)/2

In [23]:
def ryd(n_i, n_f, l):
    a = 1/(n_f**2) - 1/(n_i**2)
    return 1/(a*l)

D_df['R_value'] = ryd(D_df['n'], 2, D_df['lambda'])

e_dra = ryd(3, 2, 6.562007e-07-e_lda)-ryd(3, 2, 6.562007e-07)
e_drb = ryd(4, 2, 4.858019e-07-e_ldb)-ryd(4, 2, 4.858019e-07)
e_drc = ryd(5, 2, 4.338814e-07-e_ldc)-ryd(5, 2, 4.338814e-07)

In [24]:
R_D = (1.088970e+07 + 1.092434e+07 + 1.094371e+07)/3

e_dr = (e_dra + e_drb + e_drc)/3

In [25]:
R_D

10919250.0

In [26]:
e_dr

7828.602296501398

In [27]:
D_df

,n,H_line,primaryr,primaryl,secondaryr,secondaryl,prim_a,sec_a,lambdap,lambdas,lambda,R_value
0,3,H_a,23.327778,23.783333,51.555556,54.833333,23.669444,54.013889,6.586157e-07,6.637347e-07,6.611752e-07,1.088970e+07
1,4,H_b,17.127778,17.316667,35.711111,36.933333,17.269444,36.627778,4.870240e-07,4.893887e-07,4.882063e-07,1.092434e+07
2,5,H_c,15.316667,15.333333,31.566667,32.350000,15.329167,32.154167,4.337034e-07,4.365507e-07,4.351270e-07,1.094371e+07


## Reduced Mass Calculation ##

In [28]:
#define accepted constants
e0 = 8.8541878128e-12
h = 6.62607015e-34
c = 2.99792458e+08
e = 1.6021766208e-19

#convert to reduced mass using rydberg formula
H_rm = (R_H*8*(e0**2)*(h**3)*c)/(e**4)
D_rm = (R_D*8*(e0**2)*(h**3)*c)/(e**4)

e_hrm = ((R_H+e_hr)*8*(e0**2)*(h**3)*c)/(e**4)-(R_H*8*(e0**2)*(h**3)*c)/(e**4)
e_drm = ((R_D+e_dr)*8*(e0**2)*(h**3)*c)/(e**4)-(R_D*8*(e0**2)*(h**3)*c)/(e**4)

In [29]:
#defining accepted values
mp = 1.67262e-27
mn = 1.67493e-27

#convert reduced mass to electron mass
me_H = (1/H_rm - 1/mp)**-1
me_D = (1/D_rm - 1/(mp+mn))**-1

e_meh = (1/(H_rm+e_hrm) - 1/mp)**-1 - (1/H_rm - 1/mp)**-1
e_med = (1/(D_rm+e_drm) - 1/mp)**-1 - (1/D_rm - 1/mp)**-1

In [30]:
#finding electron mass average
me = avg(me_H, me_D)

e_me = avg(e_meh, e_med)

In [31]:
me

9.08685119833769e-31

In [32]:
e_me

8.668674361125899e-34

## Rydberg Constant for Helium ##

All done the same as for hydrogen but with no secondary lines

In [33]:
He_df = pd.DataFrame({'He_line': ['He_violet', 'He_indigo', 'He_green1', 'He_green2', 'He_orange', 'He_red1', 'He_red2'], 'primaryr': [15.5+29/90, 16.5+31/180, 17+25/60, 17.5+53/180, 20.5+19/45, 24+1/90, 25+5/36], 'primaryl': [360-(344+2/9), 360-(343+14/45), 360-(342.5+4/45), 360-(342+14/60), 360-(339+5/60), 360-(336+7/90), 360-(334.5+1/9)]})

In [34]:
e_hevr = 4.5/60
e_heir = 0.5/60
e_heg1r = 0.5/60
e_heg2r = 0.5/60
e_heor = 0.5/60
e_her1r = 0.5/60
e_her2r = 2.5/60

e_hevl = 1.5/60
e_heil = 0.5/60
e_heg1l = 0.5/60
e_heg2l = 1/60
e_heol = 0.5/60
e_her1l = 0.5/60
e_her2l = 0.5/60

In [35]:
He_df['prim_a'] = offset(He_df['primaryr'], He_df['primaryl'])

e_hev = avg(e_hevr, e_hevl)
e_hei = avg(e_heir, e_heil)
e_heg1 = avg(e_heg1r, e_heg1l)
e_heg2 = avg(e_heg2r, e_heg2l)
e_heo = avg(e_heor, e_heol)
e_her1 = avg(e_her1r, e_her1l)
e_her2 = avg(e_her2r, e_her2l)

In [36]:
He_df['lambda'] = wavelength(1/(d*1000), He_df['prim_a'], 1)

e_lhev = wavelength(1/((d-e_d)*1000), 15.788889+e_hev, 1)-wavelength(1/(d*1000), 15.788889, 1)
e_lhei = wavelength(1/((d-e_d)*1000), 16.684722+e_hei, 1)-wavelength(1/(d*1000), 16.684722, 1)
e_lheg1 = wavelength(1/((d-e_d)*1000), 17.412500+e_heg1, 1)-wavelength(1/(d*1000), 17.412500, 1)
e_lheg2 = wavelength(1/((d-e_d)*1000), 17.773611+e_heg2, 1)-wavelength(1/(d*1000), 17.773611, 1)
e_lheo = wavelength(1/((d-e_d)*1000), 20.918056+e_heo, 1)-wavelength(1/(d*1000), 20.918056, 1)
e_lher1 = wavelength(1/((d-e_d)*1000), 23.944444+e_her1, 1)-wavelength(1/(d*1000), 23.944444, 1)
e_lher2 = wavelength(1/((d-e_d)*1000), 25.326389+e_her2, 1)-wavelength(1/(d*1000), 25.326389, 1)

In [37]:
He_df

,He_line,primaryr,primaryl,prim_a,lambda
0,He_violet,15.822222,15.777778,15.788889,4.463842e-07
1,He_indigo,16.672222,16.688889,16.684722,4.710113e-07
2,He_green1,17.416667,17.411111,17.412500,4.909340e-07
3,He_green2,17.794444,17.766667,17.773611,5.007901e-07
4,He_orange,20.922222,20.916667,20.918056,5.857308e-07
5,He_red1,24.011111,23.922222,23.944444,6.658198e-07
6,He_red2,25.138889,25.388889,25.326389,7.017865e-07


In [38]:
He_rm = (1/(2*mn + 2*mp) + 1/me)**-1
e_Herm = (1/(2*mn + 2*mp) + 1/(me+e_me))**-1 - (1/(2*mn + 2*mp) + 1/me)**-1
          
R_He = (e**4)*He_rm/(8*(e0**2)*(h**3)*c)
e_rhe = (e**4)*(He_rm+e_Herm)/(8*(e0**2)*(h**3)*c) - (e**4)*He_rm/(8*(e0**2)*(h**3)*c)

In [39]:
R_He

10945101.63719641

## Predicted Balmer Series for Helium ##

In [40]:
#convert rydberg to wavelength, inverse of formula defined when converting wavelength to rydberg
def balmer(n_i, r):
    a = 1/4 - 1/(n_i**2)
    b = r*a*4
    return 1/b

#find balmer series for helium using our rydberg constant
Ba_1 = balmer(3, R_He)
Ba_2 = balmer(4, R_He)
Ba_3 = balmer(5, R_He)

e_ba1 = balmer(3, R_He-e_rhe) - balmer(3, R_He)
e_ba2 = balmer(4, R_He-e_rhe) - balmer(4, R_He)
e_ba3 = balmer(5, R_He-e_rhe) - balmer(5, R_He)

In [41]:
#actual balmer series with accepted rydberg for helium
He_1 = balmer(3, 1.09722267e+07)
He_2 = balmer(4, 1.09722267e+07)
He_3 = balmer(5, 1.09722267e+07)